In [ ]:
import optuna
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
import os
warnings.filterwarnings("ignore")

configs = [
    dict(
        symbol='XAUUSD', timeframe='H1', direction='buy', pruner_type='sucessive',
        train_start=datetime(2020,2,1), train_end=datetime(2025,2,1),
        test_start=datetime(2022,2,1),  test_end=datetime(2023,2,1),
        search_type='clusters', search_subtype='lgmm', search_filter='',
        label_method='filter', label_type='regression', n_models=1, debug=False,
    ),
]

# Crear tag para cada configuración
for cfg in configs:
    cfg["tag"] = f"{cfg['symbol']}_{cfg['timeframe']}_{cfg['direction']}_{cfg['search_type'][:2]}_{cfg.get('search_subtype', '')[:2]}_{cfg.get('search_filter', '')[:2]}_{cfg['label_method'][:2]}_{cfg['label_type'][:2]}".replace("__", "_").strip("_")

DB_FILE = f"optuna_dbs/{cfg['tag']}.db"
DB_PATH = f"sqlite:///{DB_FILE}"
STUDY_NAME = f"{cfg['tag']}"

study = None
if not os.path.exists(DB_FILE):
    study = None
else:
    try:
        study = optuna.load_study(study_name=STUDY_NAME, storage=DB_PATH)
    except Exception:
        study = None

n_trials = 5000
if study:
    n_trials = len(study.trials) + n_trials

for cfg in configs:
    cfg['n_trials'] = n_trials

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

[XAUUSD_H1_buy_cl_lg_fi_re] modelo 0 trial 1/5000 score=-1.000000 avg=4.36s mem=393.30MB 
[XAUUSD_H1_buy_cl_lg_fi_re] modelo 0 trial 2/5000 score=-1.000000 avg=5.22s mem=465.92MB 
[XAUUSD_H1_buy_cl_lg_fi_re] modelo 0 trial 3/5000 score=-1.000000 avg=4.93s mem=483.41MB 
[XAUUSD_H1_buy_cl_lg_fi_re] modelo 0 trial 4/5000 score=0.314806 avg=6.33s mem=525.04MB 
[XAUUSD_H1_buy_cl_lg_fi_re] modelo 0 trial 5/5000 score=0.314806 avg=5.79s mem=533.59MB 
[XAUUSD_H1_buy_cl_lg_fi_re] modelo 0 trial 6/5000 score=0.314806 avg=5.58s mem=566.82MB 
[XAUUSD_H1_buy_cl_lg_fi_re] modelo 0 trial 7/5000 score=0.314806 avg=5.50s mem=569.67MB 
[XAUUSD_H1_buy_cl_lg_fi_re] modelo 0 trial 8/5000 score=0.314806 avg=5.46s mem=569.29MB 
[XAUUSD_H1_buy_cl_lg_fi_re] modelo 0 trial 9/5000 score=0.314806 avg=5.25s mem=571.38MB 
[XAUUSD_H1_buy_cl_lg_fi_re] modelo 0 trial 10/5000 score=0.333249 avg=4.99s mem=573.08MB 
[XAUUSD_H1_buy_cl_lg_fi_re] modelo 0 trial 11/5000 score=0.333249 avg=5.69s mem=577.75MB 
[XAUUSD_H1_buy_c